### Imports

In [1]:
import hashlib
import io
import logging
import os
import random
import re

import contextlib2
from lxml import etree
import numpy as np
import PIL.Image
import tensorflow as tf

import argparse

from google.cloud import vision

import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline  
mpl.rc('figure', figsize=(8, 10 ))

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./vision API-06a448b64428.json"

## note 실행 : detect_document
+ 이미지에서 텍스트 감지하는 API
  (참조 : https://cloud.google.com/vision/docs/detecting-fulltext?hl=ko)

In [2]:
# Application : 12글자 이상만 표기
def detect_document(path):
    """Detects document features in an image."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.document_text_detection(image=image)
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    word_text = ''.join([symbol.text for symbol in word.symbols])
                    if len(word_text)>=12 :
                        print('상품권 : {}\t(정확도: {:.2f})'.format(word_text, word.confidence))

In [3]:
detect_document('./image/IMG_5016.jpg')

상품권 : M111012932055	(정확도: 0.98)
상품권 : M111012933400	(정확도: 0.98)
상품권 : M111012934165	(정확도: 0.98)
상품권 : M111012935917	(정확도: 0.98)


In [13]:
detect_document('./image/IMG_4674.jpg')

상품권 : M110511910779	(정확도: 0.98)
상품권 : M110511909322	(정확도: 0.98)
상품권 : M110511908953	(정확도: 0.98)
상품권 : M110511907414	(정확도: 0.94)
상품권 : M110511906630	(정확도: 0.98)
상품권 : M110511905106	(정확도: 0.95)
상품권 : M110511904394	(정확도: 0.90)
상품권 : M110511903773	(정확도: 0.95)
상품권 : M110511902910	(정확도: 0.91)
상품권 : M110511901240	(정확도: 0.97)


In [14]:
detect_document('./image/IMG_4675.jpg')

상품권 : M110511920267	(정확도: 0.98)
상품권 : M110511919394	(정확도: 0.95)
상품권 : M110511918605	(정확도: 0.93)
상품권 : M110511917657	(정확도: 0.98)
상품권 : M110511916940	(정확도: 0.97)
상품권 : M110511915853	(정확도: 0.97)
상품권 : M110511914973	(정확도: 0.98)
상품권 : M110511913453	(정확도: 0.97)
상품권 : M110511912318	(정확도: 0.96)
상품권 : M110511911540	(정확도: 0.98)


In [15]:
detect_document('./image/IMG_4676.jpg')

상품권 : M111012673930	(정확도: 0.89)
상품권 : M111012672721	(정확도: 0.98)
상품권 : M111012671119	(정확도: 0.98)
상품권 : M111012670580	(정확도: 0.90)
상품권 : M111012669099	(정확도: 0.89)
상품권 : M111012668397	(정확도: 0.98)
상품권 : M111012667891	(정확도: 0.98)
상품권 : M111012666781	(정확도: 0.98)
상품권 : M111012665364	(정확도: 0.78)
상품권 : M111012664387	(정확도: 0.98)
상품권 : M111012663129	(정확도: 0.90)
상품권 : M111012662076	(정확도: 0.87)
상품권 : M111012661730	(정확도: 0.97)
상품권 : M111012660960	(정확도: 0.95)
상품권 : M111012659197	(정확도: 0.98)


In [45]:
# Vanilla
def detect_document(path):
    """Detects document features in an image."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.document_text_detection(image=image)
#     print(response)
    img1 = cv2.imread(path)
    b, g, r = cv2.split(img1)   # img파일을 b,g,r로 분리
    img1 = cv2.merge([r,g,b])   # b, r을 바꿔서 Merge
    height, width, channels = img1.shape

    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    word_text = ''.join([symbol.text for symbol in word.symbols])
                    if len(word_text)>=12 :
#                         print('상품권 번호 : {}\t(정확도: {:.10f})'.format(word_text, word.confidence))
                        start_x, start_y = word.bounding_box.vertices[0].x, word.bounding_box.vertices[0].y
                        end_x, end_y = word.bounding_box.vertices[2].x, word.bounding_box.vertices[2].y
                        img1 = cv2.rectangle(img1, 
                                             (int(round(start_x*width,0)), int(round(start_y*height,0))), 
                                             (int(round(end_x*  width,0)), int(round(end_y  *height,0))), 
                                             (0,255,0), 3)
                        img1 = cv2.putText(img1, object_.name, (int(round(start_x*width,0)), int(round(start_y*height,0))), 
                                           cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
    
    plt.figure(figsize=(15,15))
    plt.imshow(img1)
    plt.xticks([]) # x축 눈금
    plt.yticks([]) # y축 눈금


In [46]:
detect_document('./image/200410_상품권_5만원.jpg')

ValueError: not enough values to unpack (expected 3, got 0)

In [64]:
path = "./image/200410_상품권_5만원.jpg"
img1 = cv2.imread(path)
cv2.split(img1)   # img파일을 b,g,r로 분리
# img1 = cv2.merge([r,g,b])   # b, r을 바꿔서 Merge

[]

In [65]:
print(img1)

None


In [13]:
# Vanilla
def detect_document(path):
    """Detects document features in an image."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.document_text_detection(image=image)

    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))

In [48]:
# detect_document('./image/200410_상품권_5만원.jpg')